In [236]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from math import sqrt
pd.options.mode.chained_assignment = None

In [261]:
data = pd.read_excel('C:/Users/Dmitry/Desktop/JN/University_studies/Rashetka/Data/Data.xlsx').dropna()
data = data.transpose()
data.columns = data.iloc[2]
data = data.drop(index = (['Country Code','Series Name','Series Code','Country Name']))
USA_data = data.iloc[:,:6]
NOR_data = data.iloc[:,6:]
N = len(USA_data.index)

In [262]:
for i in USA_data,NOR_data:
    i.columns = ['Y','C','I','G','Im','Ex']
for i in USA_data,NOR_data:
    for j in i.columns:
        i[j] = i[j].astype(float)

In [263]:
for i in USA_data,NOR_data:
    for j in i.columns[1:]:
        i.loc[:,j] = i.loc[:,j].astype(float)
        i.loc[:,j] = (i.loc[:,j]/100)*i.loc[:,'Y']
    i.loc[:,'Ex - Im'] = i.loc[:,'Ex'] - i.loc[:,'Im']

In [264]:
trend_lin = list(range(1,N+1))
trend_quad = [i**2 for i in trend_lin]
for i in USA_data,NOR_data:
    i.loc[:,'Linear'] = trend_lin
    i.loc[:,'Quad'] = trend_quad

In [265]:
def get_std(x,y):
    result = pd.DataFrame(columns = ['Linear','Quad','Fact','Predict','Dev'])
    result['Fact'] = y
    result[['Linear','Quad']] = x.reshape(-1,2)
    model = LinearRegression().fit(x,y)
    result['Predict'] = result['Linear']*model.coef_[0] + result['Quad']*model.coef_[1] + model.intercept_
    result['Dev'] = result['Fact'] - result['Predict']
    std = sqrt(N/(N-3))*result['Dev'].std(ddof=0)
    return std

In [270]:
x = np.array(USA_data.loc[:,['Linear','Quad']]).reshape((-1,2))
stds = pd.DataFrame(columns = ['USA','NOR'],index=range(1, 8))
stds.index = ['STDy','STDcy','STDiy','STDgy','STDmy','STDxy','STDtby']
for elem,i in enumerate([USA_data,NOR_data]):
    for k,j in enumerate(i.columns[:-3]):
        y = np.log(i.loc[:,j])
        if j == 'Y':
            stds.iloc[k,elem] = get_std(x,y)
        else:
            stds.iloc[k,elem] = get_std(x,y)/stds.iloc[0,elem]

In [271]:
corrs = pd.DataFrame(columns = ['USA','NOR'],index=range(1, 8))
corrs.index = ['CRRy','CRRcy','CRRiy','CRRgyy','CRRmy','STDxy','STDtby']

,USA,NOR
STDy,0.027456,0.029465
STDcy,1.117438,1.881741
STDiy,3.025693,3.843484
STDgy,1.693051,2.053571
STDmy,3.67849,2.726582
STDxy,4.008546,3.819132
STDtby,NaN,NaN
